[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step10.ipynb)

# 제1고지 미분 자동 계산

## 1단계 상자로서의 변수

## 2단계 변수를 낳는 함수

## 3단계 함수 연결

## 4단계 수치 미분

## 5단계 역전파 이론

## 6단계 수동 역전파

## 7단계 역전파 자동화

## 8단계 재귀에서 반복문으로

## 9단계 함수를 더 편리하게

In [1]:
mainfile = '''
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

'''

with open("main.py", 'a') as f: f.write(mainfile)

In [2]:
mainfile = '''
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        ########################################
        output = Variable(as_array(y))
        ########################################
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

'''

with open("main.py", 'a') as f: f.write(mainfile)

In [3]:
mainfile = '''
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


def square(x):
    return Square()(x)  # 한 줄로 작성


def exp(x):
    return Exp()(x)
'''

with open("main.py", 'a') as f: f.write(mainfile)

## 10단계 테스트

### 10.1 파이썬 단위 테스트

In [4]:
testfile = """
import unittest

import numpy as np

from main import Variable
from main import square


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)
""".strip()

with open("step10_1.py", 'w') as f: f.write(testfile)

In [5]:
!python -m unittest step10_1.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


### 10.2 square 함수의 역전파 테스트

In [6]:
testfile = """
import unittest

import numpy as np

from main import Variable
from main import square


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
""".strip()

with open("step10_2.py", 'w') as f: f.write(testfile)

In [7]:
!python -m unittest step10_2.py

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [8]:
testfile = '''
import unittest

import numpy as np

from main import Variable
from main import square


def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1))  # 무작위 입력값 생성
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
'''.strip()

with open("step10_3.py", 'w') as f: f.write(testfile)

In [9]:
!python -m unittest step10_3.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


### 10.4 테스트 정리

In [10]:
!python -m unittest discover -s /content -p step10_3.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
